# [LVV-T2213] - Look-up Table Application from MTMount Elevation Changes

This notebook was originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.  
It is a modified version with updated commands and simplified steps.

**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.

**Requirements:**
 - All the MT components should be enabled.
 
Please, see the [README] file for the requirements to run this notebook.  
The log messages printed in this notebook are stored in the EFD Script using the `-TTTTMMDD` format, where `TTTT` are the four last digits of the test case, `MM` is the 0-padded month, and `DD` is the 0-padded day of execution.

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2213]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2213
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

---
## Prepare the environment

Notebook preparations:
- Log on to Nublado
- Close the current repository using: `git clone https://github.com/lsst-sitcom/notebooks_vandv.git`
- You can also visit some old notebooks stored in [tickets/DM-29548/bxin/aos2comp]
- [Chronograf Dashboard]

[tickets/DM-29548/bxin/aos2comp]: https://github.com/lsst-ts/ts_notebooks/tree/tickets/DM-29548/bxin/aos2comp
[Chronograf Dashboard]: https://chronograf-summit-efd.lsst.codes/sources/1/dashboards/69?refresh=Paused&lower=now%28%29%20-%2015m

In [ ]:
test_case = "LVV-T2213"
test_exec = "LVV-EXXXX"
safe = False

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import os
import yaml

import astropy.units as u
import numpy as np

from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst.ts import utils, salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

import lsst.sitcom.vandv as vandv

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

Use the `maintel/setup_mtcs.py` script in LOVE or the [LVV-T2344] to test case and notebook to setup all the main telescope components.  
This includes simulators as well as real hardware when available (this will depend on when the test is conducted at TTS or on level 3 or on the telescope):  

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

[LVV-T2344]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2344

In [ ]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)

log = logging.getLogger("lvv-t2213")
log.level = logging.DEBUG

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()

print(f"My user_host is {domain.user_host!r}\n")

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(logging.ERROR)

In [ ]:
await mtcs.start_task

Run the cell below to hide all the messages related to the DDS and have a cleaner notebook.

In [ ]:
vandv.logger.add_filter_to_mtcs()

The `Script` CSC is used to record test checkpoints and allow to easy search in the EFD.

In [ ]:
script = salobj.Controller("Script", index=vandv.get_index(test_case))
await asyncio.sleep(10) 

script.start_task

## Check the components status

In [ ]:
# Verify that all the components have heartbeats
await mtcs.assert_liveliness()

In [ ]:
# Verify that all the components are enabled
await mtcs.assert_all_enabled()

## Setting up components

This tests runs in a special configuration where M1M3 and M2 use the mount telemetry instead of the inclinometer telemetry as inputs for the LUT.  
Such configuration allows testing their LUT without moving the mirrors physically. 

In [ ]:
az = mtcs.rem.mtmount.tel_azimuth.get()
el = mtcs.rem.mtmount.tel_elevation.get()

print(
    f"Current Az/El:\n"
    f" Az: {az.actualPosition:.2f}\n"
    f" El: {el.actualPosition:.2f}\n"
)

In [ ]:
if safe:
    await mtcs.point_azel(az=0, el=90)
else:
    await vandv.mount.moveMountInElevationSteps(mtcs.rem.mtmount, 90., azimuth=0)

### Get M1M3 LUT subscribed to the Mount

<p style="color: firebrick"><b> When M1M3 LUT is using the mount to get the elevation, avoid changes greater than 1 deg per command and avoid going lower than 82.5 deg </b></p>


- Put M1M3 in STANDBY
- Change the configuration files to make M1M3 LUT to look at the mount elevation instead of the inclinometer.
- Reset the M1M3 CSC
- Put M1M3 back in ENABLED state
- Raise the mirror, turn on FB, clear forces.
- Check that the M1M3 LUT uses the mount telemetry.

In [ ]:
await mtcs.lower_m1m3()

await mtcs.set_state(
    state=salobj.State.STANDBY,
    components=["mtm1m3"], 
    overrides = {"mtm1m3": 'Default'}
    )

<div class="alert alert-warning"> Change configuration. </div>

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mtm1m3"], 
    overrides = {"mtm1m3": 'Default'}
    )

In [ ]:
# Ensure that we are not tracking
await mtcs.stop_tracking()

# Make sure M1M3 is raised
await mtcs.raise_m1m3()

# Make sure hardpoint corrections are enabled for M1M3
await mtcs.enable_m1m3_balance_system()
await asyncio.sleep(5)

await mtcs.reset_m1m3_forces()
await asyncio.sleep(5)

### Get M2 LUT subscribed to the Mount

<p style="color: firebrick"><b> When M2 LUT is using the mount to get the elevation, avoid changes greater than 1 deg per command and avoid going lower than 82.5 deg </b></p>


See [Use of M2 EUI on Summit] for further details.

- Put M2 in STANDBY
- Change the configuration files to make M2 LUT look at the mount elevation instead of the inclinometer.
- Reset the M2 EUI.
- Put M2 back in ENABLED state
- Turn on FB, and clear forces.

[Use of M2 EUI on Summit]: https://confluence.lsstcorp.org/display/LTS/Use+of+M2+EUI+on+Summit

In [ ]:
await mtcs.set_state(
    state=salobj.State.STANDBY,
    components=["mtm2"], 
    )

<div class="alert alert-warning"> Change configuration. </div>

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mtm2"], 
    )

In [ ]:
# Make sure hardpoint corrections are enabled for M2
await mtcs.enable_m2_balance_system()
await asyncio.sleep(5)

await mtcs.reset_m2_forces()
await asyncio.sleep(5)

### Get CamHex Ready
- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure hex is at LUT position

In [ ]:
# Disable and Enable CamHex so we can assure to start fresh
await mtcs.set_state(salobj.State.DISABLED, components=["mthexapod_1"])
await mtcs.set_state(salobj.State.ENABLED, components=["mthexapod_1"])

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
# Enable compensation mode for CamHex
await mtcs.enable_compensation_mode("mthexapod_1")

In [ ]:
# Reset the Camera Hexapod position
await mtcs.reset_camera_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compensation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

### Get M2Hex Ready

- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure M2Hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_2)

In [ ]:
# Enable compensation mode for M2Hex
await mtcs.enable_compensation_mode("mthexapod_2")

In [ ]:
# Reset the M2 Hexapod position
await mtcs.reset_m2_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Elevation LUT testing 

Since we are testing only the application of the LUT to M1M3, M2, and the hexapods,  
we want to disable the active optics loop component of it.  
Depending on in what configuration you are running this test, you might want to move the mount in small or large steps. 

  * command the mount to elevation =86 deg, azimuth = 0
  * wait 39s
  * command the mount to elevation = 82 deg, azimuth = 0.
  
**Important**: We know that the minimal stable elevation angle for a non-safe operation is 82.5 deg.  
So we are adapting the test to ensure we can run it without diverging the control loop.

### Is it safe to move the mount in one big slew?  
  
If M1M3 and M2 are on Level 3 and we are running them with the LUT using the Mount Telemetry as input, it might not be safe.  
Same if M1M3 and M2 are stuck in a fixed position.  
The divergence between the actual gravity vector and what they are trying to compensate might cause the control loops to diverge causing unexpected behaviors.  
In this case, move from one position to another in small steps.  
  
Otherwhise, it is safe to move in big slews.  

In [ ]:
print(datetime.now())
script.log.info(f"START - {test_case} {test_exec} Elevation LUT testing - Safe? {safe}")

In [ ]:
if safe:
    print(datetime.now())
    script.log.info(f"MOVING - {test_case} {test_exec} Moving to az/el = 0./86.")
    await mtcs.point_azel(az=0, el=86)
    await asyncio.sleep(39.)
    
    print(datetime.now())
    script.log.info(f"MOVING - {test_case} {test_exec} Moving to az/el = 0./82.5")
    await mtcs.point_azel(az=0, el=82.5)
    await asyncio.sleep(39.)

else:
    print(datetime.now())
    script.log.info(f"MOVING - {test_case} {test_exec} Moving to az/el = 0./86.")
    await vandv.mount.moveMountInElevationSteps(mtcs.rem.mtmount, 86., azimuth=0)
    await asyncio.sleep(39.)
    
    print(datetime.now())
    script.log.info(f"MOVING - {test_case} {test_exec} Moving to az/el = 0./82.5")
    await vandv.mount.moveMountInElevationSteps(mtcs.rem.mtmount, 82.5, azimuth=0)
    await asyncio.sleep(39.)

In [ ]:
print(datetime.now())
script.log.info(f"END - {test_case} {test_exec} Elevation LUT testing - Safe? {safe}")

## Azimuth LUT testing

We need to test the application of the LUT on M1M3, M2, and the hexapods for different Azimuth angles.  
For this, we run:

* Command the mount to elevation=84 deg, azimuth=90 deg
* Wait for 39s
* Command the mount to elevation=85deg, azimuth=180deg

In [ ]:
print(datetime.now())
script.log.info(f"START - {test_case} {test_exec} Azimuth LUT testing - Safe? {safe}")

In [ ]:
if safe:
    print(datetime.now())
    script.log.info(f"MOVING - {test_case} {test_exec} Moving to az/el = 90./84.")
    await mtcs.point_azel(az=90, el=84)
    await asyncio.sleep(39.)
    
    print(datetime.now())
    script.log.info(f"MOVING - {test_case} {test_exec} Moving to az/el = 180./85.")
    await mtcs.point_azel(az=180, el=85)
    await asyncio.sleep(39.)
    
else:
    print(datetime.now())
    script.log.info(f"MOVING - {test_case} {test_exec} Moving to az/el = 90./84.")
    await vandv.mount.moveMountInElevationSteps(mtcs.rem.mtmount, 84., azimuth=90)
    await asyncio.sleep(39.)
    
    print(datetime.now())
    script.log.info(f"MOVING - {test_case} {test_exec} Moving to az/el = 180./85.")
    await vandv.mount.moveMountInElevationSteps(mtcs.rem.mtmount, 85., azimuth=180)
    await asyncio.sleep(39.)

In [ ]:
print(datetime.now())
script.log.info(f"END - {test_case} {test_exec} Azimuth LUT testing - Safe? {safe}")

### Close up

Before running this step, make sure that you put the system back in to their original state and configuration.  
This means that M1M3 and M2 need to have their LUT using their internal inclinometer.  
You can copy/paste the cells above to do that here or use LOVE to swich their states before changing configuration.   

In [ ]:
# Put the telescope back to the original position
# Specially if running at TTS
if safe:
    target = mtcs.radec_from_azel(az=0, el=80)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)
    await mtcs.stop_tracking()
    
else:
    await vandv.mount.moveMountInElevationSteps(mtcs.rem.mtmount, 80., azimuth=0.)

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
if exec_info.loc == "summit":
    await mtcs.standby()

else:
    # Bring the system back to the original state
    await mtcs.set_state(
        state=salobj.State.ENABLED,
        components=[
            "mtm1m3", 
            "mtm2",
            "mthexapod_1",
            "mthexapod_2",
            "mtaos",
        ],
        overrides={
            "mtm1m3": "Default"
        }
    )

## Wrap Up

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3", "mtm2", "mthexapod_1", "mthexapod_2"])

In [ ]:
await mtcs.set_state(state=salobj.State.ENABLED, components=["mtm2", "mthexapod_1", "mthexapod_2"])